# Preprocessing

The **UnifiedDataPipeline** class handles all data processing tasks, integrating things from our individual work:

- Load and preprocess startup data

- Integrate external data sources (market data, economic data, industry risk, regulatory data, World Governance Indicators data)

- Feature engineering:
    - Time-based features: company age, funding duration, days since last funding
    - Funding features: average round size, funding momentum
    - Location features: country risk scores (based on regulatory framework), regional risk averages (for countries not in the framework)
    - Industry features: category count, industry risk scores (based on predefined mappings), risk scores for different sectors
    - Macro features: economic indicators (GDP, inflation, unemployment)
    - URL features: domain length, HTTPS presence, URL complexity

- Preprocess data for ML:
    - Missing value imputation
    - Feature scaling
    - Outlier handling
    - Data type conversion
    - Special handling for different types of columns

In [1]:
import pandas as pd
import numpy as np
import polars as pl
from datetime import datetime, date
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
import yaml
import json
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import requests
from tqdm.auto import tqdm
import warnings
import pycountry
import zipfile
warnings.filterwarnings('ignore')

c:\Users\fabri\Desktop\uni\MSDS\2024-2025\Semester 2\Statistical Consulting\Statistical-Consulting-Case-Study-2025\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class UnifiedDataPipeline:
    # Class constants
    TODAY = pd.Timestamp(date.today())
    EXTERNAL_DATA_DIR = Path("Fabri/External Data")
    ROMAIN_DATA_DIR = Path("Romain/ExternalData")
    DATA_DIR = Path("Data")
    CACHE_DIR = Path(".cache")
    
    def __init__(self, config_path: Optional[str] = None):
        """
        Initialize the unified data pipeline
        
        Args:
            config_path: Path to configuration file (optional)
        """
        self.CACHE_DIR.mkdir(exist_ok=True)
        self.config = self._load_config(config_path)
        self.external_data = {}
        self.feature_columns = []
        self.wgi_data = None  # Store WGI data
        
    def _load_config(self, config_path: Optional[str]) -> Dict[str, Any]:
        """Load configuration from YAML file or use defaults"""
        default_config = {
            "data_sources": {
                "startup_data": str(self.ROMAIN_DATA_DIR / "startup_failures.csv"),
                "market_data": ["SP500", "NASDAQ", "DowJones", "Volatility"],
                "economic_data": ["gdp", "inflation", "unemployment"],
                "industry_data": str(self.EXTERNAL_DATA_DIR / "industry_risk_profiles.json"),
                "regulatory_data": str(self.EXTERNAL_DATA_DIR / "regulatory_framework.json")
            },
            "feature_engineering": {
                "date_features": True,
                "funding_features": True,
                "location_features": True,
                "industry_features": True,
                "macro_features": True,
                "url_features": True
            },
            "preprocessing": {
                "imputation_strategy": "median",
                "scaling": True,
                "outlier_handling": "winsorize"
            }
        }
        
        if config_path:
            with open(config_path, 'r') as f:
                config = yaml.safe_load(f)
                return {**default_config, **config}
        return default_config

    def load_external_data(self) -> None:
        """Load all required external data sources"""
        # Market data (from Fabri)
        for index in self.config["data_sources"]["market_data"]:
            try:
                df = pd.read_csv(self.EXTERNAL_DATA_DIR / f"{index}_data.csv")
                self.external_data[f"market_{index}"] = df
            except FileNotFoundError:
                print(f"Warning: {index} data not found")

        # Economic data (from Romain)
        for indicator in self.config["data_sources"]["economic_data"]:
            try:
                df = pd.read_csv(self.ROMAIN_DATA_DIR / f"{indicator}.csv")
                self.external_data[f"economic_{indicator}"] = df
            except FileNotFoundError:
                print(f"Warning: {indicator} data not found")

        # Industry and regulatory data (from Fabri)
        try:
            with open(self.EXTERNAL_DATA_DIR / "industry_risk_profiles.json", 'r') as f:
                self.external_data["industry_risk"] = json.load(f)
        except FileNotFoundError:
            print("Warning: Industry risk profiles not found")

        # Load regulatory framework
        try:
            with open(self.EXTERNAL_DATA_DIR / "regulatory_framework.json", 'r') as f:
                regulatory_data = json.load(f)
                # Ensure we have the expected structure
                if 'country_risk' not in regulatory_data:
                    regulatory_data['country_risk'] = {}
                self.external_data["regulatory_framework"] = regulatory_data
        except FileNotFoundError:
            print("Warning: Regulatory framework not found")
            self.external_data["regulatory_framework"] = {"country_risk": {}}

    def load_startup_data(self, file_path: str) -> pd.DataFrame:
        """Load and clean startup data"""
        # Define historical dates that should be treated as null values
        historical_dates = [
            '1899-12-31', '0000-00-00', '', '1871-01-01', '1869-01-01',
            '1900-01-01', '1800-01-01', '1700-01-01'
        ]
        
        # Load data using polars for better performance (from Romain)
        pl_df = pl.read_csv(
            file_path,
            try_parse_dates=True,
            null_values=historical_dates,
            dtypes={
                'founded_at': pl.Date,
                'first_funding_at': pl.Date,
                'last_funding_at': pl.Date,
                'last_milestone_at': pl.Date,
                'closed_at': pl.Date
            },
            infer_schema_length=10000,
            ignore_errors=True
        )
        
        # Convert to pandas for compatibility
        df = pl_df.to_pandas()
        
        # Basic cleaning (from MayBrghl)
        df = self._clean_funding_data(df)
        df = self._process_dates(df)
        
        return df

    def _clean_funding_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Clean funding-related columns"""
        funding_columns = [col for col in df.columns if 'funding' in col.lower()]
        
        for col in funding_columns:
            if df[col].dtype == 'object':
                # Convert to numeric, replacing non-numeric values with NaN
                df[col] = pd.to_numeric(
                    df[col].str.replace('$', '').str.replace(',', ''), 
                    errors='coerce'
                )
            
            # Handle funding_total_usd specifically
            if col == 'funding_total_usd':
                # Remove negative values
                df[col] = df[col].clip(lower=0)
                
                # Calculate IQR for outlier detection
                q1 = df[col].quantile(0.25)
                q3 = df[col].quantile(0.75)
                iqr = q3 - q1
                upper_bound = q3 + 3 * iqr  # Using 3*IQR for more conservative outlier detection
                
                # Cap outliers at upper_bound
                df[col] = df[col].clip(upper=upper_bound)
        
        return df

    def _process_dates(self, df: pd.DataFrame) -> pd.DataFrame:
        """Process date columns"""
        date_columns = [col for col in df.columns if 'date' in col.lower() or 'funding' in col.lower()]
        
        for col in date_columns:
            if df[col].dtype == 'object':  # Only process if still string
                # First try DD/MM/YYYY format
                df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')
                # If that fails, try other formats
                if df[col].isna().any():
                    df[col] = pd.to_datetime(df[col], errors='coerce')
        
        return df

    def engineer_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Engineer features from all three approaches"""
        df = df.copy()
        
        # Time-based features (from MayBrghl)
        if self.config["feature_engineering"]["date_features"]:
            df = self._add_time_features(df)
        
        # Funding features (from Fabri)
        if self.config["feature_engineering"]["funding_features"]:
            df = self._add_funding_features(df)
        
        # Location features (from Fabri)
        if self.config["feature_engineering"]["location_features"]:
            df = self._add_location_features(df)
        
        # Industry features (from Fabri)
        if self.config["feature_engineering"]["industry_features"]:
            df = self._add_industry_features(df)
        
        # Macro features (from Romain)
        if self.config["feature_engineering"]["macro_features"]:
            df = self._add_macro_features(df)
        
        # URL features (from Fabri)
        if self.config["feature_engineering"]["url_features"]:
            df = self._add_url_features(df)
        
        return df

    def _add_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add time-based features"""
        # Company age - only calculate for rows with valid founded_at dates
        valid_founded = df['founded_at'].notna()
        df.loc[valid_founded, 'company_age'] = (
            self.TODAY - df.loc[valid_founded, 'founded_at']
        ).dt.days / 365.25
        
        # Ensure company age is non-negative
        df['company_age'] = df['company_age'].clip(lower=0)
        
        # Funding duration - only calculate for rows with both first and last funding dates
        valid_funding = df['first_funding_at'].notna() & df['last_funding_at'].notna()
        df.loc[valid_funding, 'funding_duration_days'] = (
            df.loc[valid_funding, 'last_funding_at'] - 
            df.loc[valid_funding, 'first_funding_at']
        ).dt.days
        
        # Time since last funding - only calculate for rows with valid last_funding_at dates
        valid_last_funding = df['last_funding_at'].notna()
        df.loc[valid_last_funding, 'days_since_last_funding'] = (
            self.TODAY - df.loc[valid_last_funding, 'last_funding_at']
        ).dt.days
        
        return df

    def _add_funding_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add funding-related features"""
        # Average round size
        df['average_round_size'] = df['funding_total_usd'] / df['funding_rounds']
        
        # Funding momentum
        df['funding_momentum'] = df['funding_total_usd'] / df['company_age']
        
        return df

    def _add_location_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add location-based features"""
        # Combine location information
        df['location_complete'] = df[['city', 'state_code', 'country_code']].fillna('').agg(', '.join, axis=1)
        
        # Load regulatory framework
        try:
            with open(self.EXTERNAL_DATA_DIR / "regulatory_framework.json", 'r') as f:
                regulatory_data = json.load(f)
        except FileNotFoundError:
            print("Warning: Regulatory framework not found, using default values")
            regulatory_data = {}
        
        def calculate_country_risk(country_code):
            if not country_code:
                return 0.5  # Default risk for missing country codes
            
            # Convert to uppercase for case-insensitive matching
            country_code = str(country_code).upper()
            
            if country_code not in regulatory_data:
                # For countries not in our framework, use regional averages
                regional_averages = {
                    'EUROPE': 0.25,  # Lower risk for European countries
                    'ASIA': 0.35,    # Medium risk for Asian countries
                    'AFRICA': 0.45,  # Higher risk for African countries
                    'AMERICAS': 0.30, # Medium-low risk for American countries
                    'OCEANIA': 0.25  # Lower risk for Oceanian countries
                }
                
                # Try to get the region from pycountry
                try:
                    country = pycountry.countries.get(alpha_3=country_code)
                    if country:
                        # Map to our regions
                        if country.alpha_2 in ['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE']:
                            return regional_averages['EUROPE']
                        elif country.alpha_2 in ['JP', 'KR', 'CN', 'IN', 'ID', 'MY', 'PH', 'SG', 'TH', 'VN', 'TW', 'HK']:
                            return regional_averages['ASIA']
                        elif country.alpha_2 in ['NG', 'KE', 'GH', 'UG', 'TZ', 'ET', 'ZA']:
                            return regional_averages['AFRICA']
                        elif country.alpha_2 in ['US', 'CA', 'MX', 'BR', 'AR', 'CL', 'CO', 'PE']:
                            return regional_averages['AMERICAS']
                        elif country.alpha_2 in ['AU', 'NZ']:
                            return regional_averages['OCEANIA']
                except:
                    pass
                
                return 0.5  # Default risk for unknown countries
            
            country_data = regulatory_data[country_code]
            # Calculate risk score as weighted average of regulatory metrics
            # Higher regulatory quality, rule of law, and control of corruption = lower risk
            risk_score = (
                country_data['regulatory_quality'] * 0.3 +
                country_data['rule_of_law'] * 0.3 +
                country_data['control_of_corruption'] * 0.4
            )
            return 1 - risk_score  # Convert to risk score (higher score = higher risk)
        
        df['country_risk_score'] = df['country_code'].apply(calculate_country_risk)
        
        return df

    def _load_wgi_data(self) -> None:
        """Download and process World Bank's Worldwide Governance Indicators"""
        wgi_cache_file = self.CACHE_DIR / "wgi_data.csv"
        
        # Check if we have cached data
        if wgi_cache_file.exists():
            print("Loading cached WGI data...")
            try:
                self.wgi_data = pd.read_csv(wgi_cache_file)
                return
            except Exception as e:
                print(f"Error loading cached WGI data: {str(e)}")
                # If cache is corrupted, continue to download
        
        # World Bank API endpoints for WGI indicators
        indicators = {
            'Regulatory Quality': 'RQ.EST',
            'Rule of Law': 'RL.EST',
            'Control of Corruption': 'CC.EST'
        }
        
        print("Downloading WGI data from World Bank API...")
        try:
            all_data = []
            
            for indicator_name, indicator_code in indicators.items():
                # World Bank API URL
                api_url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_code}?format=json&per_page=1000&date=2020:2023"
                
                response = requests.get(api_url)
                response.raise_for_status()
                data = response.json()
                
                if not data or len(data) < 2:
                    print(f"No data found for {indicator_name}")
                    continue
                
                # Extract the data
                for entry in data[1]:
                    all_data.append({
                        'country_code': entry['countryiso3code'],
                        'year': entry['date'],
                        'indicator': indicator_name,
                        'value': entry['value']
                    })
            
            if not all_data:
                print("No WGI data found")
                self.wgi_data = None
                return
            
            # Convert to DataFrame
            df = pd.DataFrame(all_data)
            
            # Pivot the data to get indicators as columns
            self.wgi_data = df.pivot_table(
                index=['country_code', 'year'],
                columns='indicator',
                values='value'
            ).reset_index()
            
            # Normalize values from [-2.5, 2.5] to [0, 1]
            for col in indicators.keys():
                if col in self.wgi_data.columns:
                    self.wgi_data[col] = (self.wgi_data[col] + 2.5) / 5.0
            
            # Save processed data to cache
            self.wgi_data.to_csv(wgi_cache_file, index=False)
            print(f"Successfully downloaded and processed WGI data for {len(self.wgi_data)} countries")
            
        except Exception as e:
            print(f"Error downloading WGI data: {str(e)}")
            print("Using default regulatory framework instead")
            self.wgi_data = None

    def _process_wgi_data(self) -> Dict[str, Dict[str, float]]:
        """Process WGI data into our regulatory framework format"""
        if self.wgi_data is None:
            return {}
            
        try:
            # Try to find the year column (might be 'Year' or 'year')
            year_col = None
            for col in self.wgi_data.columns:
                if col.lower() == 'year':
                    year_col = col
                    break
            
            if not year_col:
                print("Warning: Year column not found in WGI data")
                return {}
            
            # Select relevant indicators
            indicators = {
                'Regulatory Quality': 'regulatory_quality',
                'Rule of Law': 'rule_of_law',
                'Control of Corruption': 'control_of_corruption'
            }
            
            # Get the latest year's data
            latest_year = self.wgi_data[year_col].max()
            latest_data = self.wgi_data[self.wgi_data[year_col] == latest_year]
            
            # Try to find country code column
            country_code_col = None
            possible_cols = ['Country Code', 'country_code', 'ISO', 'iso']
            for col in self.wgi_data.columns:
                if col in possible_cols or col.lower() in [c.lower() for c in possible_cols]:
                    country_code_col = col
                    break
            
            if not country_code_col:
                print("Warning: Country code column not found in WGI data")
                return {}
            
            regulatory_data = {}
            
            for _, row in latest_data.iterrows():
                country_code = row[country_code_col]
                if pd.isna(country_code):
                    continue
                    
                country_data = {}
                for wgi_indicator, our_indicator in indicators.items():
                    # Try different possible column names
                    value = None
                    for col in self.wgi_data.columns:
                        if wgi_indicator.lower() in col.lower():
                            value = row[col]
                            break
                    
                    if value is None or pd.isna(value):
                        continue
                        
                    # Normalize from [-2.5, 2.5] to [0, 1]
                    try:
                        value = float(value)
                        normalized_value = (value + 2.5) / 5.0
                        normalized_value = max(0, min(1, normalized_value))  # Ensure value is between 0 and 1
                        country_data[our_indicator] = normalized_value
                    except (ValueError, TypeError):
                        continue
                
                if country_data:  # Only add if we have data
                    regulatory_data[str(country_code).upper()] = country_data
            
            if not regulatory_data:
                print("Warning: No valid regulatory data found in WGI dataset")
            else:
                print(f"Successfully processed WGI data for {len(regulatory_data)} countries")
                
            return regulatory_data
            
        except Exception as e:
            print(f"Error processing WGI data: {str(e)}")
            return {}

    def _add_industry_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add industry-related features"""
        # Split categories
        df['category_list'] = df['category_list'].fillna('')
        
        # Count categories
        df['category_count'] = df['category_list'].str.count('|') + 1
        
        # Create a mapping from categories to risk scores
        industry_risk = self.external_data.get('industry_risk', {})
        risk_mapping = {
            # Technology sector
            'software': ['software', 'apps', 'application', 'saas', 'enterprise', 'programming'],
            'hardware': ['hardware', 'electronics', 'semiconductor', 'computer', 'mobile'],
            
            # Healthcare sector
            'biotech': ['biotech', 'biotechnology', 'pharmaceutical', 'life science', 'medical device'],
            'healthcare_services': ['health', 'medical', 'healthcare', 'hospital', 'wellness'],
            
            # Financial sector
            'fintech': ['fintech', 'financial technology', 'payment', 'blockchain', 'cryptocurrency'],
            'traditional_banking': ['banking', 'insurance', 'investment', 'finance'],
            
            # Retail sector
            'e_commerce': ['e-commerce', 'ecommerce', 'marketplace', 'online shopping', 'online retail'],
            'traditional_retail': ['retail', 'store', 'shopping', 'consumer goods']
        }
        
        # Define risk scores for each category
        base_risk_scores = {
            'software': 0.05,
            'hardware': 0.06,
            'biotech': 0.07,
            'healthcare_services': 0.03,
            'fintech': 0.06,
            'traditional_banking': 0.02,
            'e_commerce': 0.08,
            'traditional_retail': 0.04
        }
        
        # Function to calculate risk score for a category string
        def get_risk_score(category_string):
            if not category_string:
                return 0.05  # Default risk for empty categories
            
            categories = category_string.lower().split('|')
            matched_risks = []
            
            for cat in categories:
                cat = cat.strip()
                matched = False
                
                # Try to match the category with our defined mappings
                for industry_type, keywords in risk_mapping.items():
                    if any(keyword in cat for keyword in keywords):
                        matched_risks.append(base_risk_scores[industry_type])
                        matched = True
                        break
                
                # If no match found, assign risk based on some heuristics
                if not matched:
                    # High-risk categories
                    if any(word in cat for word in ['game', 'social', 'media', 'entertainment']):
                        matched_risks.append(0.07)  # Higher risk for consumer-facing entertainment
                    # Medium-high risk categories
                    elif any(word in cat for word in ['internet', 'digital', 'platform', 'technology']):
                        matched_risks.append(0.06)  # Technology-related
                    # Medium risk categories
                    elif any(word in cat for word in ['service', 'consulting', 'marketing']):
                        matched_risks.append(0.05)  # Service-based
                    # Medium-low risk categories
                    elif any(word in cat for word in ['education', 'training', 'research']):
                        matched_risks.append(0.04)  # Education and research
                    # Low risk categories
                    elif any(word in cat for word in ['infrastructure', 'utility', 'government']):
                        matched_risks.append(0.03)  # Infrastructure and utilities
                    else:
                        matched_risks.append(0.05)  # Default risk for unknown categories
            
            # Calculate weighted average risk score
            # Give more weight to higher risks as they're more likely to impact the company
            if matched_risks:
                return np.average(matched_risks, weights=range(1, len(matched_risks) + 1))
            return 0.05
        
        # Calculate risk scores
        df['industry_risk_score'] = df['category_list'].apply(get_risk_score)
        
        # Print summary statistics
        print("\nIndustry Risk Score Summary:")
        print(df['industry_risk_score'].describe())
        
        # Print some example mappings
        print("\nExample Category to Risk Score Mappings:")
        sample_categories = df['category_list'].sample(n=5, random_state=42)
        for cat in sample_categories:
            print(f"\nCategory: {cat}")
            print(f"Risk Score: {get_risk_score(cat)}")
        
        return df

    def _add_macro_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add macroeconomic features"""
        # Check if country_code column exists, if not try alternative names
        country_col = None
        for col in ['country_code', 'country', 'countryCode', 'country_code']:
            if col in df.columns:
                country_col = col
                break
        
        if country_col is None:
            print("Warning: No country code column found in the dataset")
            return df
            
        # Extract year from founded_at date
        df['year'] = pd.to_numeric(df['founded_at'].dt.year, errors='coerce')
            
        # Join with economic data
        for indicator in self.config["data_sources"]["economic_data"]:
            eco_data = self.external_data.get(f"economic_{indicator}")
            if eco_data is not None:
                try:
                    # Skip the first row which contains the indicator name
                    eco_data = eco_data.iloc[1:]
                    
                    # Get the first column name (usually contains country names)
                    country_column = eco_data.columns[0]
                    
                    # Convert wide format to long format
                    eco_data_long = pd.melt(
                        eco_data,
                        id_vars=[country_column],
                        var_name='year',
                        value_name=indicator
                    )
                    
                    # Clean up the data
                    eco_data_long[indicator] = eco_data_long[indicator].replace('no data', np.nan)
                    eco_data_long[indicator] = pd.to_numeric(eco_data_long[indicator], errors='coerce')
                    eco_data_long['year'] = pd.to_numeric(eco_data_long['year'], errors='coerce')
                    
                    # Remove rows with invalid years
                    eco_data_long = eco_data_long[eco_data_long['year'].notna()]
                    
                    # Create a mapping dictionary for country names to codes
                    country_mapping = {}
                    for idx, row in df.iterrows():
                        if pd.notna(row[country_col]) and pd.notna(row.get('country_name')):
                            country_mapping[row['country_name']] = row[country_col]
                    
                    # If no mapping was created, try to create one from the economic data
                    if not country_mapping:
                        # Get unique country names from both dataframes
                        eco_countries = pd.Series(eco_data_long[country_column].unique())
                        df_countries = pd.Series(df[country_col].unique())
                        unique_countries = pd.concat([eco_countries, df_countries]).unique()
                        
                        # Try to map country names to codes
                        for country in unique_countries:
                            if pd.notna(country):
                                try:
                                    # Try to get the country code from pycountry
                                    country_obj = pycountry.countries.get(name=country)
                                    if country_obj is None:
                                        country_obj = pycountry.countries.get(official_name=country)
                                    if country_obj is not None:
                                        country_mapping[country] = country_obj.alpha_3
                                except:
                                    continue
                    
                    # Map country names to codes and ensure consistent type
                    eco_data_long['country_code'] = eco_data_long[country_column].map(country_mapping)
                    eco_data_long['country_code'] = eco_data_long['country_code'].astype(str)
                    
                    # Convert the main dataframe's country_code to string type
                    df[country_col] = df[country_col].astype(str)
                    
                    # Print debug information
                    print(f"\nProcessing {indicator} data:")
                    print(f"Unique countries in economic data: {eco_data_long[country_column].nunique()}")
                    print(f"Unique country codes in economic data: {eco_data_long['country_code'].nunique()}")
                    print(f"Unique country codes in main data: {df[country_col].nunique()}")
                    
                    # Merge with the main dataframe
                    df = df.merge(
                        eco_data_long[['country_code', 'year', indicator]],
                        left_on=[country_col, 'year'],
                        right_on=['country_code', 'year'],
                        how='left'
                    )
                    
                    # Drop the redundant country_code column from the merge
                    if 'country_code_y' in df.columns:
                        df = df.drop('country_code_y', axis=1)
                    
                    # Print merge results
                    print(f"Successfully merged {indicator} data")
                    print(f"Number of non-null values: {df[indicator].notna().sum()}")
                    
                except Exception as e:
                    print(f"Warning: Failed to merge {indicator} data: {str(e)}")
                    print(f"Error details: {type(e).__name__}: {str(e)}")
                    continue
                    
        # Drop the temporary year column
        df = df.drop('year', axis=1)
                    
        return df

    def _add_url_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add URL-based features"""
        url_cols = [col for col in df.columns if 'url' in col.lower() or 'homepage' in col.lower()]
        
        if url_cols:
            url_col = url_cols[0]
            # Handle None values and convert to empty string
            df[url_col] = df[url_col].fillna('')
            
            # Add URL-based features
            df['domain_length'] = df[url_col].str.len()
            
            # More robust HTTPS detection
            df['has_https'] = df[url_col].str.lower().str.startswith('https://').fillna(False).astype(int)
            
            # More robust www detection
            df['has_www'] = df[url_col].str.lower().str.contains(r'^https?://(?:www\.)').fillna(False).astype(int)
            
            # More accurate URL complexity calculation
            df['url_complexity'] = df[url_col].str.count(r'[^/]/').fillna(0)
            # Cap complexity at a reasonable value
            df['url_complexity'] = df['url_complexity'].clip(upper=10)
            
            # Extract domain with better pattern matching
            df['domain'] = df[url_col].str.extract(r'https?://(?:www\.)?([^/]+)')
            # Clean up domain names
            df['domain'] = df['domain'].str.lower().str.strip()
        
        return df

    def preprocess_data(self, df: pd.DataFrame) -> pd.DataFrame:
        """Preprocess the data"""
        # Convert category_count to integer
        if 'category_count' in df.columns:
            df['category_count'] = df['category_count'].fillna(0).astype(int)
        
        # Get all numeric columns
        numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
        print(f"Found {len(numeric_cols)} numeric columns: {numeric_cols}")
        
        # Define columns to handle separately (not with standard imputation/scaling)
        special_handling_cols = {
            'zero_fill': [
                'funding_total_usd', 'funding_rounds', 'average_round_size', 
                'funding_momentum', 'domain_length', 'has_https', 'has_www', 
                'url_complexity', 'category_count'
            ],
            'median_fill': [
                'company_age', 'funding_duration_days', 'days_since_last_funding',
                'gdp', 'inflation', 'unemployment', 'industry_risk_score', 'country_risk_score'
            ]
        }
        
        # Handle zero-fill columns
        for col in special_handling_cols['zero_fill']:
            if col in df.columns:
                df[col] = df[col].fillna(0)
                if col in ['funding_rounds', 'category_count', 'has_https', 'has_www']:
                    df[col] = df[col].astype(int)
        
        # Handle median-fill columns
        for col in special_handling_cols['median_fill']:
            if col in df.columns:
                df[col] = df[col].fillna(df[col].median())
        
        # Get remaining numeric columns for standard processing
        special_cols = [col for group in special_handling_cols.values() for col in group]
        standard_numeric_cols = [col for col in numeric_cols if col not in special_cols and col != 'country_risk_score']
        
        if standard_numeric_cols:
            print(f"Processing standard numeric columns: {standard_numeric_cols}")
            numeric_df = df[standard_numeric_cols].copy()
            
            # Handle missing values
            imputer = SimpleImputer(strategy=self.config["preprocessing"]["imputation_strategy"])
            imputed_values = imputer.fit_transform(numeric_df)
            
            # Scale numeric features
            if self.config["preprocessing"]["scaling"]:
                scaler = StandardScaler()
                scaled_values = scaler.fit_transform(imputed_values)
                numeric_df = pd.DataFrame(
                    scaled_values,
                    columns=standard_numeric_cols,
                    index=numeric_df.index
                )
                
                # Handle outliers
                if self.config["preprocessing"]["outlier_handling"] == "winsorize":
                    numeric_df = self._winsorize_outliers(numeric_df, standard_numeric_cols)
                
                # Update the original DataFrame
                df[standard_numeric_cols] = numeric_df
        
        # Handle missing values in location data
        location_cols = ['state_code', 'region', 'city']
        for col in location_cols:
            if col in df.columns:
                df[col] = df[col].fillna('Unknown')
        
        # Handle missing values in domain
        if 'domain' in df.columns:
            df['domain'] = df['domain'].fillna('')
        
        # Handle missing values in dates
        date_cols = ['founded_at', 'first_funding_at', 'last_funding_at']
        for col in date_cols:
            if col in df.columns:
                if col == 'founded_at':
                    # For founded_at, use a default date if missing
                    df[col] = df[col].fillna(pd.Timestamp('2000-01-01'))
                else:
                    # For funding dates, use the last available date
                    df[col] = df[col].fillna(df[col].max())
        
        return df

    def _winsorize_outliers(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """Winsorize outliers in specified columns"""
        for col in columns:
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            df[col] = df[col].clip(lower_bound, upper_bound)
        
        return df

    def process(self, startup_data_path: str) -> pd.DataFrame:
        """
        Complete data processing pipeline
        
        Args:
            startup_data_path: Path to startup data file
            
        Returns:
            Processed DataFrame ready for modeling
        """
        # Load external data
        self.load_external_data()
        
        # Load and clean startup data
        df = self.load_startup_data(startup_data_path)
        
        # Engineer features
        df = self.engineer_features(df)
        
        # Preprocess data
        df = self.preprocess_data(df)
        
        return df

In [3]:
if __name__ == "__main__":
    # Example usage
    pipeline = UnifiedDataPipeline()
    processed_data = pipeline.process(str(UnifiedDataPipeline.ROMAIN_DATA_DIR / "startup_failures.csv"))
    
    # Save processed data in both formats
    parquet_path = "processed_data.parquet"
    csv_path = "processed_data.csv"
    
    # Save as parquet
    processed_data.to_parquet(parquet_path, index=False)
    print(f"\nProcessed data saved to {parquet_path} (parquet format)")
    
    # Save as CSV
    processed_data.to_csv(csv_path, index=False)
    print(f"Processed data saved to {csv_path} (CSV format)")
    
    # Print summary statistics
    print("\nSummary of processed data:")
    print(f"Total rows: {len(processed_data):,}")
    print(f"Total columns: {len(processed_data.columns):,}")
    print("\nNumeric columns summary:")
    numeric_cols = processed_data.select_dtypes(include=['float64', 'int64']).columns
    print(processed_data[numeric_cols].describe())
    
    print("\nAvailable features:")
    for col in processed_data.columns:
        print(f"- {col}") 


Industry Risk Score Summary:
count    66368.000000
mean         0.052947
std          0.010010
min          0.020000
25%          0.050000
50%          0.050000
75%          0.058000
max          0.080000
Name: industry_risk_score, dtype: float64

Example Category to Risk Score Mappings:

Category: Agriculture|Apps|Information Technology|Internet|Mobile|Social Entrepreneurship|Telecommunications
Risk Score: 0.05857142857142857

Category: Software
Risk Score: 0.05

Category: Finance
Risk Score: 0.02

Category: Health and Wellness
Risk Score: 0.03

Category: Electronics
Risk Score: 0.06

Processing gdp data:
Unique countries in economic data: 227
Unique country codes in economic data: 174
Unique country codes in main data: 138
Successfully merged gdp data
Number of non-null values: 45543

Processing inflation data:
Unique countries in economic data: 228
Unique country codes in economic data: 174
Unique country codes in main data: 138
Successfully merged inflation data
Number of non-null

# Machine Learning: Credit Risk

The **CreditRiskML** class handles all ML tasks for credit risk assessment:

- Feature preparation:
    - Financial: total funding, funding rounds, average funding per round, frequency of funding rounds, funding growth rate, funding efficiency relative to company age, funding stability
    - Temporal: company age, duration of funding period, time since last funding round, ratio of time since last funding to company age, recency of funding events, rate of funding rounds
    - Geographic: country-specific risk score, GDP, inflation rate, unemployment rate, combined economic stability
    - Industry: industry-specific risk score, number of business categories, industry concentration
    - Operational: URL complexity, HTTPS presence, WWW presence, domain name length, combined digital presence score

- Model training and evaluation:
    - Random Forest with class weights to handle class imbalance (weights are inversely proportional to their frequencies in the input data, giving more importance to the minority class, namely the "default" class)
    - Stratified 5-fold cross-validation (model was trained on 4/5 of the data and evaluated on 1/5 of the data, with the final model trained on the entire dataset)
    - Performance metrics: precision, recall, F1 score, ROC AUC, confusion matrix

- Risk and credit decisions:
    - Risk category: low, medium, high, very high
    - Credit decision: approve, approve with conditions, review, decline
    - Credit limits: none, low, medium, high
    - Monitoring frequency: none, weekly, monthly, quarterly

- Visualisations

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    roc_auc_score, precision_recall_curve, classification_report,
    roc_curve, precision_score, recall_score, f1_score,
    average_precision_score, accuracy_score, confusion_matrix
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import xgboost as xgb
import lightgbm as lgb
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.ensemble import IsolationForest
import joblib
import json
from datetime import datetime
import logging
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import shap
import warnings
import os
import gc
warnings.filterwarnings('ignore')

In [17]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class CreditRiskML:
    def __init__(self, model_dir: str = "models"):
        """
        Initialize the ML credit risk assessment system
        
        Args:
            model_dir: Directory to save trained models
        """
        self.model_dir = Path(model_dir)
        self.model_dir.mkdir(exist_ok=True)
        
        # Define feature groups
        self.feature_groups = {
            'financial': [
                'funding_total_usd',
                'funding_rounds',
                'average_round_size',
                'funding_momentum'
            ],
            'temporal': [
                'company_age',
                'funding_duration_days',
                'days_since_last_funding'
            ],
            'geographic': [
                'country_risk_score',
                'gdp',
                'inflation',
                'unemployment'
            ],
            'industry': [
                'industry_risk_score',
                'category_count'
            ],
            'operational': [
                'url_complexity',
                'has_https',
                'has_www',
                'domain_length'
            ]
        }
        
        # Initialize models
        self.model = None
        self.scaler = None
        
    def prepare_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepare features for model training
        
        Args:
            df: Input DataFrame with raw features
            
        Returns:
            Processed DataFrame with prepared features
        """
        # Create feature groups
        for group, features in self.feature_groups.items():
            # Check if all features exist
            missing_features = [f for f in features if f not in df.columns]
            if missing_features:
                logger.warning(f"Missing features for {group}: {missing_features}")
                continue
            
            # Create group-specific features
            if group == 'financial':
                df['funding_efficiency'] = df['funding_total_usd'] / (df['company_age'] + 1)
                df['funding_stability'] = df['funding_rounds'] / (df['company_age'] + 1)
            
            elif group == 'temporal':
                df['funding_recency'] = 1 / (df['days_since_last_funding'] + 1)
                df['funding_velocity'] = df['funding_rounds'] / df['funding_duration_days']
            
            elif group == 'geographic':
                df['economic_stability'] = (df['gdp'] - df['inflation']) / (df['unemployment'] + 1)
            
            elif group == 'industry':
                df['industry_concentration'] = 1 / (df['category_count'] + 1)
            
            elif group == 'operational':
                df['digital_presence'] = (
                    df['has_https'].astype(int) + 
                    df['has_www'].astype(int) + 
                    (df['url_complexity'] > 0).astype(int)
                ) / 3
        
        return df
    
    def train_models(self, X: pd.DataFrame, y: pd.Series):
        """Train models with memory optimization"""
        try:
            # Clear memory
            gc.collect()
            
            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            
            # Train model
            logger.info("Training model...")
            model = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )
            model.fit(X_train_scaled, y_train)
            
            # Store models and scalers
            self.model = model
            self.scaler = scaler
            
            # Calculate feature importance
            self.feature_importance = dict(zip(
                X.columns,
                model.feature_importances_
            ))
            
            # Save models
            self._save_models()
            
            # Evaluate model
            y_pred = model.predict(X_test_scaled)
            logger.info("\nClassification Report:")
            logger.info(classification_report(y_test, y_pred))
            
            return True
            
        except Exception as e:
            logger.error(f"Error during model training: {str(e)}")
            return False
    
    def predict_risk(self, X: pd.DataFrame) -> pd.DataFrame:
        """Predict risk scores for new data"""
        try:
            if self.model is None or self.scaler is None:
                raise ValueError("Model or scaler not initialized. Please train the model first.")
                
            # Scale features
            X_scaled = self.scaler.transform(X)
            
            # Get predictions
            risk_scores = self.model.predict_proba(X_scaled)[:, 1]
            
            # Create risk categories
            risk_categories = pd.cut(
                risk_scores,
                bins=[0, 0.2, 0.5, 0.8, 1],
                labels=['Low', 'Medium', 'High', 'Very High']
            )
            
            # Create output DataFrame
            results = pd.DataFrame({
                'risk_score': risk_scores,
                'risk_category': risk_categories
            })
            
            return results
            
        except Exception as e:
            logger.error(f"Error during prediction: {str(e)}")
            return None
    
    def make_credit_decisions(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Make credit decisions based on risk predictions
        
        Args:
            X: Input DataFrame with preprocessed numeric features
            
        Returns:
            DataFrame with credit decisions
        """
        # Get risk predictions
        risk_predictions = self.predict_risk(X)
        if risk_predictions is None:
            return None
            
        # Define decision rules
        def get_credit_decision(row):
            if row['risk_category'] == 'Low':
                return {
                    'decision': 'APPROVE',
                    'credit_limit': 'HIGH',
                    'monitoring_frequency': 'QUARTERLY'
                }
            elif row['risk_category'] == 'Medium':
                return {
                    'decision': 'APPROVE_WITH_CONDITIONS',
                    'credit_limit': 'MEDIUM',
                    'monitoring_frequency': 'MONTHLY'
                }
            elif row['risk_category'] == 'High':
                return {
                    'decision': 'REVIEW',
                    'credit_limit': 'LOW',
                    'monitoring_frequency': 'WEEKLY'
                }
            else:  # Very High
                return {
                    'decision': 'DECLINE',
                    'credit_limit': 'NONE',
                    'monitoring_frequency': 'NONE'
                }
        
        # Apply decision rules
        decisions = risk_predictions.apply(get_credit_decision, axis=1)
        decisions_df = pd.DataFrame(decisions.tolist())
        
        # Combine with risk predictions
        results = pd.concat([risk_predictions, decisions_df], axis=1)
        
        return results
    
    def _save_models(self):
        """Save trained models to disk"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        model_path = self.model_dir / f"rf_{timestamp}.joblib"
        joblib.dump(self.model, model_path)
        logger.info(f"Saved model to {model_path}")
        
        # Convert feature importance values to float64 for JSON serialization
        importance_dict = {
            'feature_importance': {
                str(k): float(v) for k, v in self.feature_importance.items()
            }
        }
        
        # Save feature importance
        importance_path = self.model_dir / f"feature_importance_{timestamp}.json"
        with open(importance_path, 'w') as f:
            json.dump(importance_dict, f)
        logger.info(f"Saved feature importance to {importance_path}")
    
    def load_models(self, timestamp: str):
        """
        Load trained models from disk
        
        Args:
            timestamp: Timestamp of the models to load
        """
        for name in ['xgb', 'cox', 'anomaly']:
            model_path = self.model_dir / f"{name}_{timestamp}.joblib"
            if model_path.exists():
                self.model = joblib.load(model_path)
                logger.info(f"Loaded {name} model from {model_path}")
        
        # Load feature importance
        importance_path = self.model_dir / f"feature_importance_{timestamp}.json"
        if importance_path.exists():
            with open(importance_path, 'r') as f:
                self.feature_importance = json.load(f)
            logger.info(f"Loaded feature importance from {importance_path}")
    
    def evaluate_models(self, X: pd.DataFrame, y: pd.Series) -> dict:
        """
        Evaluate model performance
        
        Args:
            X: Test data features
            y: True labels
            
        Returns:
            Dictionary with evaluation metrics
        """
        try:
            # Get predictions
            predictions = self.predict_risk(X)
            
            # Calculate metrics
            metrics = {
                'auc_roc': roc_auc_score(y, predictions['risk_score']),
                'risk_distribution': predictions['risk_category'].value_counts(normalize=True).to_dict()
            }
            
            # Calculate confusion matrix
            y_pred = (predictions['risk_score'] > 0.5).astype(int)
            cm = confusion_matrix(y, y_pred)
            
            # Create confusion matrix visualization
            plt.figure(figsize=(10, 8))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=['Non-Default', 'Default'],
                        yticklabels=['Non-Default', 'Default'])
            plt.title('Confusion Matrix')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            
            # Ensure metrics directory exists
            metrics_dir = Path("output/metrics")
            metrics_dir.mkdir(parents=True, exist_ok=True)
            
            # Save confusion matrix plot
            cm_path = metrics_dir / "confusion_matrix.png"
            plt.savefig(cm_path, bbox_inches='tight', dpi=300)
            plt.close()
            logger.info(f"Saved confusion matrix to {cm_path}")
            
            # Print classification report
            logger.info("\nClassification Report:")
            logger.info(classification_report(
                y,
                predictions['risk_score'] > 0.5
            ))
            
            # Print confusion matrix
            logger.info("\nConfusion Matrix:")
            logger.info(cm)
            
            # Print risk distribution
            logger.info("\nRisk Distribution:")
            for category, proportion in metrics['risk_distribution'].items():
                logger.info(f"{category}: {proportion:.2%}")
            
            return metrics
            
        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")
            raise

    def plot_risk_distribution(self, predictions: pd.DataFrame, save_path: str = None):
        """
        Plot the distribution of risk scores and categories
        
        Args:
            predictions: DataFrame with risk predictions
            save_path: Optional path to save the plot
        """
        fig = make_subplots(rows=2, cols=1, subplot_titles=('Risk Score Distribution', 'Risk Category Distribution'))
        
        # Risk score distribution
        fig.add_trace(
            go.Histogram(x=predictions['risk_score'], name='Risk Score'),
            row=1, col=1
        )
        
        # Risk category distribution
        category_counts = predictions['risk_category'].value_counts()
        fig.add_trace(
            go.Bar(x=category_counts.index, y=category_counts.values, name='Risk Categories'),
            row=2, col=1
        )
        
        fig.update_layout(height=800, title_text="Risk Distribution Analysis")
        if save_path:
            fig.write_html(save_path)
        return fig
    
    def plot_feature_importance(self, save_path: str = None):
        """
        Plot feature importance from the XGBoost model
        
        Args:
            save_path: Optional path to save the plot
        """
        if 'xgb' not in self.feature_importance:
            logger.warning("No feature importance data available")
            return None
        
        importance_data = pd.DataFrame({
            'feature': list(self.feature_importance['xgb'].keys()),
            'importance': list(self.feature_importance['xgb'].values())
        }).sort_values('importance', ascending=True)
        
        fig = px.bar(
            importance_data,
            x='importance',
            y='feature',
            orientation='h',
            title='Feature Importance'
        )
        
        if save_path:
            fig.write_html(save_path)
        return fig
    
    def plot_shap_summary(self, X: pd.DataFrame, save_path: str = None):
        """
        Create SHAP summary plot to explain model predictions
        
        Args:
            X: Feature matrix
            save_path: Optional path to save the plot
        """
        if 'xgb' not in self.model:
            logger.warning("No trained XGBoost model available")
            return None
        
        # Calculate SHAP values
        explainer = shap.TreeExplainer(self.model)
        shap_values = explainer.shap_values(X)
        
        # Create summary plot
        plt.figure(figsize=(10, 8))
        shap.summary_plot(shap_values, X, plot_type="bar")
        plt.title("SHAP Feature Importance")
        
        if save_path:
            plt.savefig(save_path)
            plt.close()
        else:
            plt.show()
    
    def plot_risk_by_feature(self, df: pd.DataFrame, feature: str, save_path: str = None):
        """
        Plot risk scores against a specific feature
        
        Args:
            df: Input DataFrame
            feature: Feature to plot against
            save_path: Optional path to save the plot
        """
        predictions = self.predict_risk(df)
        combined = pd.concat([df[[feature]], predictions], axis=1)
        
        fig = px.scatter(
            combined,
            x=feature,
            y='risk_score',
            color='risk_category',
            title=f'Risk Score by {feature}',
            labels={'risk_score': 'Risk Score', feature: feature}
        )
        
        if save_path:
            fig.write_html(save_path)
        return fig
    
    def create_risk_dashboard(self, df: pd.DataFrame, save_dir: str = "risk_dashboard"):
        """
        Create a comprehensive risk dashboard
        
        Args:
            df: Input DataFrame
            save_dir: Directory to save dashboard components
        """
        save_dir = Path(save_dir)
        save_dir.mkdir(exist_ok=True)
        
        # Get predictions
        predictions = self.predict_risk(df)
        
        # Create various plots
        self.plot_risk_distribution(
            predictions,
            save_path=str(save_dir / "risk_distribution.html")
        )
        
        self.plot_feature_importance(
            save_path=str(save_dir / "feature_importance.html")
        )
        
        # Plot risk by key features
        for feature in ['funding_total_usd', 'company_age', 'country_risk_score']:
            if feature in df.columns:
                self.plot_risk_by_feature(
                    df,
                    feature,
                    save_path=str(save_dir / f"risk_by_{feature}.html")
                )
        
        # Create SHAP summary plot
        X = self.prepare_features(df)
        self.plot_shap_summary(
            X,
            save_path=str(save_dir / "shap_summary.png")
        )
        
        # Save predictions
        predictions.to_csv(save_dir / "risk_predictions.csv", index=False)
        
        logger.info(f"Risk dashboard created in {save_dir}")

    def explain_prediction(self, df: pd.DataFrame, index: int) -> dict:
        """
        Explain the prediction for a specific instance
        
        Args:
            df: Input DataFrame
            index: Index of the instance to explain
            
        Returns:
            Dictionary with explanation details
        """
        if 'xgb' not in self.model:
            logger.warning("No trained XGBoost model available")
            return None
        
        # Prepare features
        X = self.prepare_features(df)
        instance = X.iloc[[index]]
        
        # Get SHAP values
        explainer = shap.TreeExplainer(self.model)
        shap_values = explainer.shap_values(instance)
        
        # Get prediction details
        prediction = self.predict_risk(df.iloc[[index]])
        
        # Create explanation
        explanation = {
            'risk_score': float(prediction['risk_score'].iloc[0]),
            'risk_category': prediction['risk_category'].iloc[0],
            'feature_contributions': dict(zip(
                X.columns,
                shap_values[0]  # For binary classification, we use the first class
            )),
            'base_value': float(explainer.expected_value[0]),
            'instance_features': instance.iloc[0].to_dict()
        }
        
        return explanation

In [18]:
if __name__ == "__main__":
    try:
        # Load data with memory optimization
        logger.info("Loading data...")
        df = pd.read_parquet(
            "output/processed_data.parquet",
            columns=[
                'country_risk_score', 'industry_risk_score', 'funding_total_usd', 
                'status', 'name', 'country_code', 'company_age', 'funding_rounds',
                'days_since_last_funding', 'category_count', 'funding_duration_days'
            ]
        )
        
        # Calculate default status from status column
        df['default_status'] = (df['status'] == 'closed').astype(int)
        
        # Create more informative features
        df['funding_per_round'] = df['funding_total_usd'] / (df['funding_rounds'] + 1)
        df['funding_frequency'] = df['funding_rounds'] / (df['funding_duration_days'] + 1)
        df['time_since_funding_ratio'] = df['days_since_last_funding'] / (df['company_age'] + 1)
        
        # Select features for modeling
        feature_cols = [
            'country_risk_score', 'industry_risk_score', 'funding_total_usd',
            'company_age', 'funding_rounds', 'days_since_last_funding',
            'category_count', 'funding_duration_days', 'funding_per_round',
            'funding_frequency', 'time_since_funding_ratio'
        ]
        
        # Prepare feature matrix
        X = df[feature_cols].copy()
        y = df['default_status']
        
        # Handle missing values with median imputation
        for col in X.columns:
            X[col] = X[col].fillna(X[col].median())
        
        # Calculate class weights
        n_samples = len(y)
        n_classes = len(np.unique(y))
        class_weights = dict(zip(
            np.unique(y),
            n_samples / (n_classes * np.bincount(y))
        ))
        
        logger.info("Class weights: %s", class_weights)
        
        # Initialize ML system
        ml_system = CreditRiskML()
        
        # Initialize model with balanced class weights
        model = RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_leaf=5,
            class_weight=class_weights,
            random_state=42,
            n_jobs=-1
        )
        
        # Perform cross-validation
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = {
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': [],
            'roc_auc': [],
            'avg_precision': []
        }
        
        # Initialize confusion matrix for all folds
        total_cm = np.zeros((2, 2))
        
        logger.info("Performing cross-validation...")
        
        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_val_scaled = scaler.transform(X_val)
            
            # Train model
            model.fit(X_train_scaled, y_train)
            
            # Get predictions
            y_pred = model.predict(X_val_scaled)
            y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
            
            # Calculate confusion matrix for this fold
            cm = confusion_matrix(y_val, y_pred)
            total_cm += cm
            
            # Calculate metrics
            cv_scores['accuracy'].append(accuracy_score(y_val, y_pred))
            cv_scores['precision'].append(precision_score(y_val, y_pred, zero_division=0))
            cv_scores['recall'].append(recall_score(y_val, y_pred))
            cv_scores['f1'].append(f1_score(y_val, y_pred))
            cv_scores['roc_auc'].append(roc_auc_score(y_val, y_pred_proba))
            cv_scores['avg_precision'].append(average_precision_score(y_val, y_pred_proba))
            
            logger.info(f"\nFold {fold} Results:")
            logger.info(classification_report(y_val, y_pred))
            logger.info("\nConfusion Matrix:")
            logger.info(cm)
        
        # Create and save confusion matrix visualization for all folds
        plt.figure(figsize=(10, 8))
        sns.heatmap(total_cm.astype(int), annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Non-Default', 'Default'],
                    yticklabels=['Non-Default', 'Default'])
        plt.title('Aggregated Confusion Matrix (All Folds)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        
        # Save confusion matrix plot
        metrics_dir = Path("output/metrics")
        metrics_dir.mkdir(parents=True, exist_ok=True)
        cm_path = metrics_dir / "confusion_matrix.png"
        plt.savefig(cm_path, bbox_inches='tight', dpi=300)
        plt.close()
        logger.info(f"\nSaved aggregated confusion matrix to {cm_path}")
        
        # Print cross-validation results
        logger.info("\nCross-validation Results:")
        for metric, scores in cv_scores.items():
            logger.info(f"{metric}: {np.mean(scores):.3f} (+/- {np.std(scores):.3f})")
        
        # Train final model on full dataset
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        model.fit(X_scaled, y)
        
        # Store model and scaler in ml_system
        ml_system.model = model
        ml_system.scaler = scaler
        
        # Make predictions and credit decisions
        credit_decisions = ml_system.make_credit_decisions(X)
        
        # Add company identifiers back to predictions
        credit_decisions['company_name'] = df['name']
        credit_decisions['country'] = df['country_code']
        credit_decisions['status'] = df['status']
        
        # Save predictions
        output_dir = Path("output")
        output_dir.mkdir(exist_ok=True)
        predictions_path = output_dir / "ml_credit_risk_predictions.csv"
        credit_decisions.to_csv(predictions_path, index=False)
        logger.info(f"Saved predictions to {predictions_path}")
        
        # Create performance metrics visualizations
        metrics_dir = output_dir / "metrics"
        metrics_dir.mkdir(exist_ok=True)
        
        # ROC Curve
        fig_roc = go.Figure()
        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
            X_val_scaled = scaler.transform(X.iloc[val_idx])
            y_val = y.iloc[val_idx]
            y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
            
            fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
            auc_score = roc_auc_score(y_val, y_pred_proba)
            
            fig_roc.add_trace(
                go.Scatter(x=fpr, y=tpr, name=f'Fold {fold} (AUC = {auc_score:.3f})')
            )
        
        fig_roc.add_trace(
            go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Random', line=dict(dash='dash'))
        )
        
        fig_roc.update_layout(
            title='ROC Curves across CV Folds',
            xaxis_title='False Positive Rate',
            yaxis_title='True Positive Rate',
            showlegend=True
        )
        fig_roc.write_html(str(metrics_dir / "roc_curves.html"))
        
        # Precision-Recall Curve
        fig_pr = go.Figure()
        for fold, (train_idx, val_idx) in enumerate(cv.split(X, y), 1):
            X_val_scaled = scaler.transform(X.iloc[val_idx])
            y_val = y.iloc[val_idx]
            y_pred_proba = model.predict_proba(X_val_scaled)[:, 1]
            
            precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
            avg_precision = average_precision_score(y_val, y_pred_proba)
            
            fig_pr.add_trace(
                go.Scatter(x=recall, y=precision, name=f'Fold {fold} (AP = {avg_precision:.3f})')
            )
        
        fig_pr.update_layout(
            title='Precision-Recall Curves across CV Folds',
            xaxis_title='Recall',
            yaxis_title='Precision',
            showlegend=True
        )
        fig_pr.write_html(str(metrics_dir / "precision_recall_curves.html"))
        
        # Feature Importance Plot
        importance_data = pd.DataFrame({
            'feature': feature_cols,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=True)
        
        fig_imp = px.bar(
            importance_data,
            x='importance',
            y='feature',
            orientation='h',
            title='Feature Importance'
        )
        fig_imp.write_html(str(metrics_dir / "feature_importance.html"))
        
        # Print summary
        print("\nPredictions Summary:")
        print("=" * 80)
        print(f"Total companies analyzed: {len(credit_decisions)}")
        print("\nRisk Distribution:")
        print(credit_decisions['risk_category'].value_counts(normalize=True).mul(100).round(2))
        print("\nCredit Decision Distribution:")
        print(credit_decisions['decision'].value_counts(normalize=True).mul(100).round(2))
        print("\nSample of predictions (first 5 companies):")
        print(credit_decisions[['company_name', 'country', 'status', 'risk_score', 'risk_category', 'decision', 'credit_limit', 'monitoring_frequency']].head())
        
        print("\nPerformance visualizations have been saved to:", str(metrics_dir))
        logger.info("ML processing completed!")
            
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        import traceback
        traceback.print_exc() 

2025-04-29 12:25:24,825 - __main__ - INFO - Loading data...
2025-04-29 12:25:24,922 - __main__ - INFO - Class weights: {np.int64(0): np.float64(0.5518709462830533), np.int64(1): np.float64(5.31965373517153)}
2025-04-29 12:25:24,926 - __main__ - INFO - Performing cross-validation...
2025-04-29 12:25:30,663 - __main__ - INFO - 
Fold 1 Results:
2025-04-29 12:25:30,679 - __main__ - INFO -               precision    recall  f1-score   support

           0       0.94      0.86      0.90     12026
           1       0.28      0.50      0.36      1248

    accuracy                           0.83     13274
   macro avg       0.61      0.68      0.63     13274
weighted avg       0.88      0.83      0.85     13274

2025-04-29 12:25:30,679 - __main__ - INFO - 
Confusion Matrix:
2025-04-29 12:25:30,679 - __main__ - INFO - [[10376  1650]
 [  621   627]]
2025-04-29 12:25:36,486 - __main__ - INFO - 
Fold 2 Results:
2025-04-29 12:25:36,501 - __main__ - INFO -               precision    recall  f1-scor


Predictions Summary:
Total companies analyzed: 66368

Risk Distribution:
risk_category
Low          49.38
Medium       30.94
High         17.53
Very High     2.16
Name: proportion, dtype: float64

Credit Decision Distribution:
decision
APPROVE                    49.38
APPROVE_WITH_CONDITIONS    30.94
REVIEW                     17.53
DECLINE                     2.16
Name: proportion, dtype: float64

Sample of predictions (first 5 companies):
             company_name country     status  risk_score risk_category  \
0                   #fame     IND  operating    0.106189           Low   
1                :Qounter     USA  operating    0.129574           Low   
2  (THE) ONE of THEM,Inc.    None  operating    0.316706        Medium   
3                 0-6.com     CHN  operating    0.422246        Medium   
4        004 Technologies     USA  operating    0.051348           Low   

                  decision credit_limit monitoring_frequency  
0                  APPROVE         HIGH       

Results:

- Accuracy: The model correctly classifies about 83% of all cases

- Precision: When the model predicts a company will default, it's correct about 28% of the time

- Recall: The model correctly classifies about 52% of actual default cases

- F1: The model achieves an F1 score of 36%, meaning it sacrifices precision to achieve better recall

- ROC AUC: The model achieves a ROC AUC of 80%, indicating good discrimination ability

- Confusion matrix:
    - For non-defaulting companies, the model achieves high precision (94%), meaning it's usually correct when it predicts a company won't default, and good recall (86%), meaning it correctly identifies most safe companies
    - For defaulting companies, the model achieves low precision (28%), meaning many companies predicted to default won't actually default, and moderate recall (50%), meaning it catches about half of the actual default cases


Conclusion:

- The model is better at risk detection than precision

- The model is better at identifying potentially risky companies (high recall) but makes more false positive predictions (low precision), hence might be appropriate for initial risk screening, where it's more important to catch potential risks than to be precise in every prediction

- The model is stable and not overfitting